# PART 2 - FUNDAMENTAL STOCK ANALYSIS

Fundamental analysis is done to determine whether a stock is likely to cause <b>Profit or Loss</b> based on indicators like P/E ratio, Debt/Eq ratio, ROE, Insider own etc. 

The entire procedure in this notebook consists of the following steps: 

- Step 1 - Function to scrape <a>finviz.com</a> and return all the metrics required for doing fundamental analysis
- Step 2 - Function to iterate through each of the stocks, collect their metrics from <a>finviz.com</a> and store in lists
- Step 3 - Function to clean, preprocess metrics data and store them in a dataframe
- Step 4 - Function to filter the metrics dataframe which includes only the required attributes for analysis
- Step 5 - Perform the Fundamental Analysis
- Step 6 - Results and Inferences
<br>

<br>

<br>


In [7]:
#import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib.request as urllib2
import pandas as pd
import json
import os
import time
import csv
from ipykernel import kernelapp as app

stock_list = ['AAPL','IBM','GS','AMZN','GE','GOOG']

### Step 1 - Function to scrape <a>finviz.com</a> and return all the metrics required for doing fundamental analysis
<br>

In [3]:
def scrape_finviz(symbol):
    
    #specify the url
    quote_page = "http://finviz.com/quote.ashx?t=" + str(symbol.lower()) 
    
    #query the website and return the html to the variable ‘page’
    page = urllib2.urlopen(quote_page)
    
    #parse the html using beautiful soup and store in variable `soup`
    soup = BeautifulSoup(page, 'html.parser')
    
    metrics = [s.get_text(separator="\n", strip=True) for s in soup.find_all( 'tr', attrs={'class' : 'table-dark-row'})]
    
    print(metrics)
    
    return metrics

### Step 2 - Function to iterate through each of the stocks, collect their metrics from <a>finviz.com</a> and store in lists
<br>

In [ ]:
def collect_metrics():
    
    all_stock_metrics = {}
    for stock in stock_list:
        all_metrics = []
        all_values = []
        stock = str(stock)
        metrics = scrape_finviz(stock)
        for elem in metrics:
            elems = elem.split('\n')
            for e in range(0,len(elems),2):

                #contains all metric names
                all_metrics.append(elems[e])

                #contains corresponding values
                all_values.append(elems[e + 1])
                
        #contains names and values 
        all_stock_metrics[stock] = [all_metrics, all_values]
        
        return all_stock_metrics

### Step 3 - Function to clean, preprocess metrics data and store them in a dataframe
<br>

In [ ]:
def preprocess_metrics_data():
    
    all_stock_metrics = collect_metrics()
    
    comp = ["Company"]+ all_stock_metrics['AAPL'][0]
    metrics_df = pd.DataFrame(columns = comp)

    company_names = ["Apple", "IBM","Goldman Sachs","Amazon","General Electric","Google"]
    companies_dict = {}

    for i in range(0,len(company_names)):
        companies_dict[company_names[i]] = [company_names[i]] + all_stock_metrics[stock_list[i]][1]
        metrics_df = metrics_df.append(pd.Series(companies_dict[company_names[i]], index = comp ), ignore_index= True)

    print(metrics_df)

In [292]:
preprocess_metrics_data()

,Company,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,...,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
0,Apple,DJIA S&P500;,14.20,11.94,0.07%,4.73B,-1.06%,801.39B,13.30,12.74,...,Jan 29 AMC,23.00%,41.85M,170.89,2.20,5.88%,5.56%,-10.78%,"22,168,778",0.86%
1,IBM,DJIA S&P500;,11.38,11.77,0.10%,911.20M,-0.89%,122.09B,9.46,14.17,...,Jan 22 AMC,97.20%,5.47M,136.05,2.70,4.21%,10.54%,-0.25%,"3,311,382",1.54%
2,Goldman Sachs,DJIA S&P500;,7.96,24.05,0.40%,374.69M,-3.23%,71.69B,7.17,26.68,...,Jan 16 BMO,24.00%,4.45M,194.49,2.40,-0.67%,6.05%,-10.38%,"2,382,563",1.65%
3,Amazon,S&P; 500,80.22,19.83,16.10%,499.29M,-2.59%,794.37B,40.12,39.66,...,Jan 31 AMC,0.00%,6.90M,1638.01,1.70,-0.53%,1.79%,-4.89%,"4,812,832",2.95%
4,General Electric,S&P; 500,-,-2.40,0.15%,8.92B,-1.76%,89.47B,10.95,0.92,...,Jan 31 BMO,-,129.63M,9.98,2.60,5.82%,19.57%,-13.48%,"66,568,943",-0.50%
5,Google,S&P; 500,25.06,43.70,-,680.92M,-3.34%,745.61B,23.27,47.06,...,Feb 04 AMC,-,1.80M,1121.37,1.70,2.73%,5.21%,0.25%,"1,603,207",2.41%


### Step 4 - Function to filter the metrics dataframe which includes only the required attributes for analysis
<br>


In [297]:
#Make a list of all the columns required in the analysis 
columns = ['Company', 'P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

#Replace all the hyphens'-' with '000' as hyphens will cause computation problems 
metrics_df = metrics_df.replace(to_replace=r'-', value= '000', regex=True)

#Create a new dataframe with only the required columns
new_metrics_df = metrics_df[columns].copy()

new_metrics_df

,Company,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
0,Apple,6.81,14.20,13.30,1.09,0.97,11.94,1.72%,50.90%,26.60%,0004.80%,0.07%
1,IBM,6.17,11.38,9.46,11.86,2.37,11.77,4.69%,31.10%,17.40%,0.70%,0.10%
2,Goldman Sachs,0.98,7.96,7.17,1.24,7.08,24.05,1.67%,6.80%,1.50%,25.20%,0.40%
3,Amazon,17.90,80.22,40.12,1.83,1.13,19.83,000,27.00%,11.90%,166.60%,16.10%
4,General Electric,2.77,000,10.95,000,3.66,0002.40,0.40%,000,0001.90%,000937.50%,0.15%
5,Google,4.29,25.06,23.27,1.53,000,43.70,000,000,000,000,000


### Step 5 - Perform the Fundamental Analysis
<br>


#### 1. Businesses which are quoted at low valuations
P/E < 20 <br>
P/B < 3
<br>

In [298]:
new_metrics_df1 = new_metrics_df[(new_metrics_df['P/E'].astype(float)<20) & (new_metrics_df['P/B'].astype(float) < 3)]
new_metrics_df1

,Company,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
2,Goldman Sachs,0.98,7.96,7.17,1.24,7.08,24.05,1.67%,6.80%,1.50%,25.20%,0.40%
4,General Electric,2.77,000,10.95,000,3.66,0002.40,0.40%,000,0001.90%,000937.50%,0.15%




#### 2. Businesses which have demonstrated earning power
EPS Q/Q > 10% <br> 

In [299]:
new_metrics_df['EPS Q/Q'] = new_metrics_df['EPS Q/Q'].map(lambda x: x[:-1])
new_metrics_df

,Company,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
0,Apple,6.81,14.20,13.30,1.09,0.97,11.94,1.72%,50.90%,26.60%,0004.80,0.07%
1,IBM,6.17,11.38,9.46,11.86,2.37,11.77,4.69%,31.10%,17.40%,0.70,0.10%
2,Goldman Sachs,0.98,7.96,7.17,1.24,7.08,24.05,1.67%,6.80%,1.50%,25.20,0.40%
3,Amazon,17.90,80.22,40.12,1.83,1.13,19.83,000,27.00%,11.90%,166.60,16.10%
4,General Electric,2.77,000,10.95,000,3.66,0002.40,0.40%,000,0001.90%,000937.50,0.15%
5,Google,4.29,25.06,23.27,1.53,000,43.70,000,000,000,00,000


In [300]:
new_metrics_df2 = new_metrics_df[new_metrics_df['EPS Q/Q'].astype(float) > 10]
new_metrics_df2

,Company,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
2,Goldman Sachs,0.98,7.96,7.17,1.24,7.08,24.05,1.67%,6.80%,1.50%,25.20,0.40%
3,Amazon,17.90,80.22,40.12,1.83,1.13,19.83,000,27.00%,11.90%,166.60,16.10%
4,General Electric,2.77,000,10.95,000,3.66,0002.40,0.40%,000,0001.90%,000937.50,0.15%


#### 3. Businesses earning good returns on equity while employing little or no debt
Debt/Eq < 1 <br>
ROE > 10%

In [301]:
new_metrics_df['ROE'] = new_metrics_df['ROE'].map(lambda x: x[:-1])

In [303]:
new_metrics_df3 = new_metrics_df[(new_metrics_df['Debt/Eq'].astype(float) < 1) & (new_metrics_df['ROE'].astype(float) > 10)]
new_metrics_df3

,Company,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
0,Apple,6.81,14.20,13.30,1.09,0.97,11.94,1.72%,50.90,26.60%,0004.80,0.07%


#### 4. Management having substantial ownership in the business
Insider own > 30%

In [304]:
new_metrics_df['Insider Own'] = new_metrics_df['Insider Own'].map(lambda x: x[:-1])

In [305]:
new_metrics_df4 = new_metrics_df[new_metrics_df['Insider Own'].astype(float) > 30]
new_metrics_df4

,Company,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own


### Step 6 - Results and Inferences


#### Companies likely to cause profit in the Portfolio

- Google
- Goldman Sachs 
- Amazon 
- Apple

#### Companies likely to cause loss in the portfolio

- IBM 
- General Electric